In [ ]:
"""\
# Copyright (C) 2024 Jesús Bautista Villar <jesbauti20@gmail.com>
"""
!python3 --version

In [1]:
import numpy as np
import os

# Animation tools
from IPython.display import HTML
from matplotlib.animation import PillowWriter

# Import the Swarm Systems Lab Simulator
from ssl_simulator import SimulationEngine, create_dir, load_sim, add_src_to_path
from ssl_simulator.robot_models import Unicycle2D
from ssl_simulator.visualization import set_paper_parameters
set_paper_parameters(fontsize=15)

from ssl_simulator.components.gvf import GvfEllipse

# -------------------------------------------------------------------------------------
# Import custom functions and classes (src in path is requiered)
add_src_to_path("", deep=0)

from sim_core.visualization import PlotBasicGvf
from sim_core.visualization import AnimationXY, AnimationXY, AnimationXYPhi
from sim_core.controllers import GvfIK

# -------------------------------------------------------------------------------------
# Data and output directories
DATA_DIR = os.path.join("..", "data")
OUTPUT_DIR = os.path.join("..", "output")
SIMDATA_FILE = os.path.join(OUTPUT_DIR, "data.csv")

create_dir(OUTPUT_DIR)

The directory '..\output' already exists!


## ICRA video animations

### Animation of the simulation

In [2]:
# Define the initial state
N = 1

p = np.ones((N,2)) * np.array([[150,-200]])
speed = np.ones((N)) * np.array([14])
theta = np.ones((N)) * np.array([-90])

x0 = [p, speed, theta]

# Controller settings
A = np.ones((N)) * 0.3
omega = np.ones((N)) * np.pi/4

# --------------------------------
# Generate the trajectory to be followed
a, b = 100, 70
XYoff, alpha = [0, 0], 0

gvf_traj = GvfEllipse(XYoff,alpha,a,b)

# Select the robot model
robot_model = Unicycle2D(x0)

# Initialize the controller
s, ke, kn = 1, 0.35, 1
controller = GvfIK(gvf_traj, s, ke, kn, A, omega)


# Then, construct the Simulator Engine
dt = 0.01
dt_log = 0.1
simulator = SimulationEngine(robot_model, controller, time_step=dt, 
                             log_filename=SIMDATA_FILE, log_time_step=dt_log)

# --------------------------------
# Run the simulation and save data
tf = 10

simulator.run(tf)
# --------------------------------

Running simulation: 100%|██████████| 1000/1000 [00:01<00:00, 880.67it/s]


In [3]:
# Load previously saved simulation data from the specified file
simulation_data = load_sim(SIMDATA_FILE, debug=False)

# Initialize the animator with the loaded simulation data
animator = AnimationXY(
    gvf_traj, simulation_data, 
    dpi=100, figsize=(6,6), xlims=[-200,200], ylims=[-250,150])

anim = animator.gen_animation(fps=1/dt_log)
HTML(anim.to_html5_video()) # It takes a loooot of time...

Simulating 99 frames... 
Progress:


100%|██████████| 99/99 [00:12<00:00,  9.62it/s]